In [6]:
# Setup
import numpy as np
import scipy as sp
import pandas as p
from IPython.display import display
from sklearn import datasets, linear_model, preprocessing
from sklearn.metrics import mean_squared_error, r2_score

In [18]:
# Test
years = ['2011']
         #, '2012', '2013', '2014', '2015', '2016', '2017']

train_x = p.DataFrame()
train_y = p.DataFrame()
test_x = p.DataFrame()
test_y = p.DataFrame()

for year in years:
    
    # Get battings stats
    batting_df = p.read_csv(year + 'Batting.csv', 
                            skipfooter=2, 
                            engine='python')

    
    batting_df['Name'] = batting_df.Name.str.replace("\\" , '#')
    batting_df['Name'] = batting_df.Name.str.replace("#'a0" , ' ')
    batting_df['Name'] = batting_df['Name'].str.split('#').str[0]

#     batting_df['Name'] = batting_df['Name'].str.split('\').str[-1]
#     batting_df['Name'] = batting_df.Name.str.replace("'a0" , ' ')     

    display(batting_df)
    batting_df.to_csv('testimport', index=False)

 
    salaries_df = p.read_csv(year + 'Salary.csv', 
                             engine='python')
    salaries_df['Name'] = salaries_df['Name'].str.split('\\').str[-1]
    
    batting_df['Pos'] = batting_df['Pos'].str[:-1]
    salaries_df['Pos'] = salaries_df['Pos'].str[:-1]
    

    # take out NAN batting averages
    batting_df = batting_df[np.isfinite(batting_df['BA'])]
    # don't care about pitchers batting
    batting_df = batting_df[batting_df.Pos != '1']
    batting_df = batting_df[~batting_df.Pos.str.startswith('/1')]
   
    batting_df = batting_df[batting_df.Name.isin(salaries_df.Name)]
    salaries_df = salaries_df[salaries_df.Name.isin(batting_df.Name)]
    salaries_df = salaries_df.sort_values('Name')
            
    if year == years[0]:
        train_x = batting_df
        train_y = salaries_df
    elif year != years[-1]:
        train_x = p.concat([train_x, batting_df])
        train_y = p.concat([train_y, salaries_df]) 
    else: 
        test_x = batting_df
        test_y = salaries_df
        
train_x = train_x.fillna(0)
test_x = test_x.fillna(0)
train_y = train_y.fillna(0)
test_y = test_y.fillna(0)

#display(train_x)

#just player ID's
train_ID = train_y.filter(['Name'], axis=1)

# take out team, league
train_x.drop(train_x.columns[[0, 1, 2, 3, 4, 29]], axis=1, inplace=True)
test_x.drop(test_x.columns[[0, 1, 2, 3, 4, 29]], axis=1, inplace=True)

#display(train_x)

# get salary out of salaries
train_y = train_y.filter(['Salary'], axis=1)
test_y = test_y.filter(['Salary'], axis=1)

#drop $
train_y['Salary'] = train_y['Salary'].str.replace('$','')
test_y['Salary'] = test_y['Salary'].str.replace('$','')

#display(train_y)

# display(train_x)

# check = train_x.values[95]
# print train_y.values[95]
# display(train_y)

,Rk,Name,Age,Tm,Lg,G,PA,AB,R,H,...,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,Pos
0,1,Fernando Abad*,25,HOU,NL,28,0,0,0,0,...,NaN,NaN,NaN,0,0,0,0,0,0,1\
1,2,Bobby Abreu*,37,LAA,AL,142,585,502,54,127,...,0.365,0.717,105.0,183,8,1,1,3,5,D79\
2,3,Juan Abreu,26,HOU,NL,7,0,0,0,0,...,NaN,NaN,NaN,0,0,0,0,0,0,/1\
3,4,Jeremy Accardo,29,BAL,AL,1,0,0,0,0,...,NaN,NaN,NaN,0,0,0,0,0,0,1\
4,5,Alfredo Aceves,28,BOS,AL,2,0,0,0,0,...,NaN,NaN,NaN,0,0,0,0,0,0,1\
5,6,Dustin Ackley*,23,SEA,AL,90,376,333,39,91,...,0.417,0.766,120.0,139,3,0,0,3,1,4/D3\
6,7,Manny Acosta,30,NYM,NL,41,0,0,0,0,...,NaN,NaN,NaN,0,0,0,0,0,0,1\
7,8,Mike Adams,32,TOT,MLB,46,0,0,0,0,...,NaN,NaN,NaN,0,0,0,0,0,0,1\
8,9,Ryan Adams,24,BAL,AL,29,96,89,9,25,...,0.326,0.659,81.0,29,7,1,0,0,0,4\
9,10,Nathan Adcock,23,KCR,AL,1,0,0,0,0,...,NaN,NaN,NaN,0,0,0,0,0,0,1\


IndexError: index 0 is out of bounds for axis 1 with size 0

In [9]:
# #start regression
# regr = linear_model.LinearRegression()
# regr.fit(train_x.values, train_y.values)

# # salary_predict(testingYear)
# print('Weights = \n', regr.coef_)
# predicted = regr.predict(test_x.values)
# # print test_y.values
# test_y.Salary = test_y.Salary.astype(float)

# # Explained variance score: 1 is perfect prediction
# print('Variance score: %.2f' % r2_score(test_y.values, predicted))

# diff = predicted - (test_y.values)
# binarizer = preprocessing.Binarizer().fit(diff)
# # classifier on 
# over_under_binary = binarizer.transform(diff)
# display(over_under_binary)

# # overpaid = np.where((predicted - (test_y.values)) < -10000000)[0]
# # for over in overpaid: 
# #     display(test_x.iloc[over])
    


In [12]:
['ARI','ATL','BAL','BOS','CHC','CHW','CIN','CLE','COL','DET','HOU','KC','LAA','LAD','MIA','MIL','MIN','NYM','NYY','OAK','PHI','PIT','SD','SEA','SF','STL','TB','TEX','TOR','WSH']

TypeError: cannot concatenate a non-NDFrame object

In [19]:
#try to read testimport       
test_df = p.read_csv('testimport', 
                            engine='python')
display(test_df)


,Rk,Name,Age,Tm,Lg,G,PA,AB,R,H,...,SLG,OPS,OPS+,TB,GDP,HBP,SH,SF,IBB,Pos
0,1,Fernando Abad*,25,HOU,NL,28,0,0,0,0,...,NaN,NaN,NaN,0,0,0,0,0,0,1\
1,2,Bobby Abreu*,37,LAA,AL,142,585,502,54,127,...,0.365,0.717,105.0,183,8,1,1,3,5,D79\
2,3,Juan Abreu,26,HOU,NL,7,0,0,0,0,...,NaN,NaN,NaN,0,0,0,0,0,0,/1\
3,4,Jeremy Accardo,29,BAL,AL,1,0,0,0,0,...,NaN,NaN,NaN,0,0,0,0,0,0,1\
4,5,Alfredo Aceves,28,BOS,AL,2,0,0,0,0,...,NaN,NaN,NaN,0,0,0,0,0,0,1\
5,6,Dustin Ackley*,23,SEA,AL,90,376,333,39,91,...,0.417,0.766,120.0,139,3,0,0,3,1,4/D3\
6,7,Manny Acosta,30,NYM,NL,41,0,0,0,0,...,NaN,NaN,NaN,0,0,0,0,0,0,1\
7,8,Mike Adams,32,TOT,MLB,46,0,0,0,0,...,NaN,NaN,NaN,0,0,0,0,0,0,1\
8,9,Ryan Adams,24,BAL,AL,29,96,89,9,25,...,0.326,0.659,81.0,29,7,1,0,0,0,4\
9,10,Nathan Adcock,23,KCR,AL,1,0,0,0,0,...,NaN,NaN,NaN,0,0,0,0,0,0,1\
